# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
!gdown --id '1kLJD-TYQT7caGhc-kwMQTA6XvcJpSW5K' --output data.zip 
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1kLJD-TYQT7caGhc-kwMQTA6XvcJpSW5K
To: /content/data.zip
372MB [00:02, 141MB/s]
Archive:  data.zip
replace timit_11/train_11.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
data.zip  model.ckpt  prediction.csv  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.1

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 85

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

867

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 3000)
        self.layer2 = nn.Linear(3000, 1024)
        self.layer3 = nn.Linear(1024, 512)
        self.layer4 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39) 
        self.dropout = nn.Dropout(p=0.2)
        self.batch_normalize_3000 = nn.BatchNorm1d(3000)
        self.batch_normalize_1024 = nn.BatchNorm1d(1024)
        self.batch_normalize_512 = nn.BatchNorm1d(512)
        self.batch_normalize_128 = nn.BatchNorm1d(128)
        self.act_fn = nn.LeakyReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.batch_normalize_3000(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer2(x)
        x = self.batch_normalize_1024(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer3(x)
        x = self.batch_normalize_512(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer4(x)
        x = self.batch_normalize_128(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.out(x)
        
        return x

In [ ]:
import torch
import torch.nn as nn
m = nn.Dropout(p=0.2)
input = torch.randn(5, 2)
print(len(input))
output = m(input)
print(output)

5
tensor([[ 0.8067, -0.5095],
        [ 0.7837, -0.4482],
        [ 0.0000,  1.0086],
        [ 0.0674,  0.5947],
        [-0.0880,  0.0000]])


## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 25               # number of training epoch
learning_rate = 0.1       # learning rate
L2_regluation =0.0001
# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)#, weight_decay=L2_regluation,)

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/025] Train Acc: 0.516240 Loss: 1.661711 | Val Acc: 0.656894 loss: 1.153090
saving model with acc 0.657
[002/025] Train Acc: 0.609319 Loss: 1.331384 | Val Acc: 0.685220 loss: 1.036103
saving model with acc 0.685
[003/025] Train Acc: 0.636412 Loss: 1.228564 | Val Acc: 0.704839 loss: 0.965011
saving model with acc 0.705
[004/025] Train Acc: 0.651714 Loss: 1.172966 | Val Acc: 0.710230 loss: 0.942201
saving model with acc 0.710
[005/025] Train Acc: 0.663297 Loss: 1.127455 | Val Acc: 0.707620 loss: 0.941315
[006/025] Train Acc: 0.671589 Loss: 1.097580 | Val Acc: 0.718198 loss: 0.912360
saving model with acc 0.718
[007/025] Train Acc: 0.678177 Loss: 1.072492 | Val Acc: 0.714848 loss: 0.924171
[008/025] Train Acc: 0.683817 Loss: 1.050523 | Val Acc: 0.718092 loss: 0.908687
[009/025] Train Acc: 0.688711 Loss: 1.031546 | Val Acc: 0.725255 loss: 0.886878
saving model with acc 0.725
[010/025] Train Acc: 0.693405 Loss: 1.014364 | Val Acc: 0.724409 loss: 0.892239


KeyboardInterrupt: ignored

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))